# Table Length Measurements

### Authors: 
- Troels C. Petersen (Niels Bohr Institute)
- Christian Michelsen (Niels Bohr Institute)
- Étienne Bourbeau (Niels Bohr Institute)


### Date:    
- 21-11-2019 (latest update)

***

Python program for analysing measurements of the length of the lecture table in Auditorium A at NBI.  
There are two measurements each with estimated error of the table length:
1. Measurement with a 30cm ruler.
2. Measurement with a 2m folding ruler.

Each person was asked not only to state the measurement, but also their estimated uncertainty. None of the persons could see others measurements in order to get the largest degree of independence. Also, the 30cm ruler measurement was asked to be done first.

***

In [ ]:
# First, import the modules you want to use:
# Comment the line below on or off if you want to activate/deactivate interactive plotting
%matplotlib notebook

import numpy as np                                     # Matlab like syntax for linear algebra and functions
import matplotlib.pyplot as plt                        # Plots and figures like you know them from Matlab
from iminuit import Minuit                             # The actual fitting tool, better than scipy's
import sys
from scipy import stats
from scipy.special import erfc                         # Error function, to get integral of Gaussian

And add out our custom functions:

In [ ]:
sys.path.append('../../../External_Functions')
from ExternalFunctions import Chi2Regression, BinnedLH, UnbinnedLH
from ExternalFunctions import nice_string_output, add_text_to_ax # useful functions to print fit results on figure

Options for the program: 

In [ ]:
blinded = True
save_plots = False

r = np.random          # Random numbers
r.seed(42)             # We set the numbers to be random, but the same for each run

## Functions:

A Gaussian PDF (unit integral):

In [ ]:
# Define your PDF / model 
def gauss_pdf(x, mu, sigma):
    """Normalized Gaussian"""
    return 1 / np.sqrt(2 * np.pi) / sigma * np.exp(-(x - mu) ** 2 / 2. / sigma ** 2)

Extended Gaussian PDF (including normalisation, which enables fit to histograms):

In [ ]:
def gauss_extended(x, N, mu, sigma) :
    """Non-normalized Gaussian"""
    return N * gauss_pdf(x, mu, sigma)

## Initial data analysis:

Before we even look at the look at the data, we decide whether or not we want to blind the analysis by adding a constant to all measurements:


In [ ]:
if blinded:
    blinding = r.normal(0, 0.1)      # I add a constant (Gaussian with +-10cm) to remain "blind"
else:
    blinding = 0

Define what datafiles we want to look at. Extend it to suit your analysis: 

In [ ]:
infiles = ["data_TableMeasurements2019.txt",
           "data_TableMeasurements2018.txt",
           "data_TableMeasurements2017.txt",
           "data_TableMeasurements2016.txt",
           "data_TableMeasurements2015.txt",
           "data_TableMeasurements2014.txt",
           "data_TableMeasurements2013.txt",
           "data_TableMeasurements2012.txt",
           "data_TableMeasurements2011.txt",
           "data_TableMeasurements2010.txt",
           "data_TableMeasurements2009.txt"]

We read in all the data from the `infiles` files:

In [ ]:
L30cm = np.array([])
eL30cm = np.array([])
L2m = np.array([])
eL2m = np.array([])

# Loop over files and open them
for infile in infiles:
    
    tmp_L30cm, tmp_eL30cm, tmp_L2m, tmp_eL2m = np.loadtxt(infile, skiprows=2, unpack=True)
    
    L30cm = np.append(L30cm, tmp_L30cm + blinding)
    eL30cm = np.append(eL30cm, tmp_eL30cm)
    L2m = np.append(L2m, tmp_L2m + blinding)
    eL2m = np.append(eL2m, tmp_eL2m)
    
N_read  = len(L30cm)       # Number of measurements read in total


print(f"\n\nRead all {len(infiles)} file(s) which included {N_read} measurements. \n")

Now we plot the data. Using the interactive window, you can move the plot and zoom in and out of interesting regions. To Stop the interactive view of the plot, you can click on the blue power button.

In [ ]:
Nbins = 1000
minL = 0.0
maxL = 5.0
binwidth = (maxL-minL)/Nbins

# Define two histograms with all the lengths recorded:
fig_raw, ax = plt.subplots(nrows=2, figsize=(9,14), gridspec_kw={'hspace':0.5})
ax_L30cm, ax_L2m = ax

hist_L30cm = ax_L30cm.hist(L30cm, bins=Nbins, range=(minL, maxL), histtype='step', label='Binned Data')
ax_L30cm.set_title('Length estimates by 30cm ruler',fontsize=15)
ax_L30cm.set_ylabel('Frequency',fontsize=15)
ax_L30cm.set_xlabel('Table length (m)',fontsize=15)

hist_L2m = ax_L2m.hist(L2m, bins=Nbins, range=(minL, maxL), histtype='step', label='Binned Data')
ax_L2m.set_title('Length estimates by 2m ruler',fontsize=15)
ax_L2m.set_ylabel('Frequency',fontsize=15)
ax_L2m.set_xlabel('Table length (m)',fontsize=15)

fig_raw.tight_layout()

## 30 cm ruler:

We consider the measurements for the 30cm ruler and focus on that for now. Below is a mean and RMS calculation along with a general Gaussian fit to all the data. Somehow, it doesn't seem optimal/right...

In [ ]:
print(f"  Initial estimate of raw data (30 cm ruler):  {L30cm.mean():.3f} +- {L30cm.std(ddof=1)/np.sqrt(len(L30cm)):.3f} m     (RMS = {L30cm.std(ddof=1):.3f})")

### Fitting the data:
Then we set up an __unbinned likelihood fit__, and fit the full data set (note that we set the initial step size, called error_mu and error_sigma, and fix the normalisation):

In [ ]:
ullh_L30cm = UnbinnedLH(gauss_extended, L30cm, weights=None)
minuit_L30cm = Minuit(ullh_L30cm, pedantic=False, N=len(L30cm)*binwidth, fix_N=True,
                      mu=L30cm.mean(), error_mu=0.01,
                      sigma=L30cm.std(ddof=1), error_sigma=0.01, print_level=0) 
minuit_L30cm.migrad();

# The fitted values of the parameters and their uncertainties:
L30cm_fit_mu = minuit_L30cm.values["mu"]
L30cm_fit_mu_error = minuit_L30cm.errors["mu"]
L30cm_fit_sigma = minuit_L30cm.values["sigma"]
print(f"  The unbinned LH fitted estimate (30cm ruler): {L30cm_fit_mu:.3f} +- {L30cm_fit_mu_error:.3f} m     RMS = {L30cm_fit_sigma:.3f}")

### Plotting:

Now we plot the fit on the figure:

In [ ]:
# Create the x-axis for the plot of the fitted function
xaxis = np.linspace(minL, maxL, 10000) 

# Compute the fitted function for x_fit
yaxis = gauss_extended(xaxis, *minuit_L30cm.args) 

# Plot the fitted function
ax_L30cm.plot(xaxis, yaxis, '-', label='Fit')
ax_L30cm.legend()

# Show the actual fit and fit results:
fig_raw

### Next steps:

For the above, you should by now be aware, that there is definitely room for improvement. It is now up to you to select in (based on solid quantified arguments) and/or correct the data, and subsequently produce a better estimate of the length of the table in world famous Auditorium A. Good luck...

In [ ]:
# Clean up in the data:
L30cm = np.sort(L30cm)
mask = (L30cm>=np.percentile(L30cm,0))*(L30cm<=np.percentile(L30cm,100))      # Write your own data selection!!!
L30cm_subset = L30cm[mask]
print(f"  The number of measurements used before and after is: {len(L30cm):d} and {len(L30cm_subset):d}")

***

Start by taking a close look at the data, first by inspecting the numbers in the data file (yeps, open the damn thing, and look over the numbers!), and then by considering the histograms produced by running the notebook. 

To begin with, only consider the 30cm ruler measurements, and disregard the estimated/guessed uncertainties. You can then expand from there, as guided below by questions.


# Questions:

1. Consider the mean and width. Is the result as you would expect it? And do you think that it is close to the best possible (i.e. most accurate and precise) estimate? NOTE: Make sure that you know the difference between accuracy and precision!!! See "Common definition" in: http://en.wikipedia.org/wiki/Accuracy_and_precision

2. Do any of the measurements looks wrong/bad/suspicious? Do you see any repeated mistakes done for obvious reasons? Would you correct or exclude any of the measurements and how would you justify this? This problem requires that you discuss internally, and then each do what you think most justified/best. Apply this to the list of measurements, and perhaps produce a new list with your accepted and possibly corrected measurements in (to save the original data). How many measurements did you throw away in the end?

3. Fit your accepted length measurements with a Gaussian distribution, possibly in a (small?) range around what you believe is the true value. What would be your strategy for choosing a range? Is the Gaussian distribution justified? Also, do you see any "human" effects? Did any of your class mates (or you?) not read to mm precision?

4. Once you have selected the measurements you want to use, calculate the mean, RMS and uncertainty on the mean. How much did your result improve in precision?

5. Is the number of mismeasurements 30cm below the actual length more common than those 30cm above? Determine the ratio $r_{mis} = N_{30cm~high} / N_{30cm~low}$ including both its statistical uncertainty, and the systematic uncertainty of defining exactly when a measurement is 30cm off.



#### Now repeat the above for the 2m folding rule
...and consider all the questions below as optional problems, where you might just answer a few for the 30cm data!
 
***


6. How much better/worse is the single measurement uncertainty from the 30cm ruler case to the 2m folding rule?

7. The "Pull" distribution is defined as the plot of $z_i = \left(x_i - \mu \right)/\sigma_i$ where $\mu$ is the *sample* mean of $x$, and $x_i$ and $\sigma_i$ are the single measurements and their corresponding uncertainties. If the measurements and uncertainties are good, then it should give a unit Gaussian. Is that the case? And thus, were the uncertainty estimates/guesses reasonable? If not, then the pull distribution is often used to remove overly precise measurements (those far out on the tails), and afterwards to scale the errors on the remaining measurements to a reasonable level. Consider the pull distribution, and see if any measurements have suspicious uncertainties.

8. Try to calculate the weighted mean. Did you get a good Chi2 probability, when doing so? Do you need to discard more dubious measurements? Did the result improve further in precision?

9. Does the length of the table seems to be different when measured with a 30cm and a 2m ruler? Quantify this statement! I.e. what is the difference, and what is the uncertainty on that difference? Is this significantly away from 0?

10. If you were asked for the best estimate of the length of the table, what would you do? (If posssible, read Bevington page 58 bottom!)


### Not too advanced questions:
11. Consider the 2018 and 2019 data files with additional information in (Gender and measurement speed), and find out if gender and measurement speed (i.e. measured calmly before course start or in hast during course start) have an impact on the quality of the measurements.


### Advanced questions:
12. Is there any correlation between the errors on the measurements and the distance value? I.e. do you see any effect of those measuring e.g. too long having a smaller/larger uncertainty? What would the effect of this be?